# Information management
## word2vec을 이용한 기억속 영화 찾기(doc2vec)
## 1. Read Data

In [375]:
import numpy as np
import pandas as pd
import os, re
#from konlpy.tag import Twitter
#from konlpy.utils import pprint
#from gensim.models.doc2vec import LabeledSentence, Doc2Vec
#from gensim.models import Word2Vec
#from sklearn.feature_extraction.text import TfidfVectorizer

In [396]:
# change directory
os.chdir('D:/UNIST/17-2학기 자료/IM')
# read data
df=pd.read_csv('merged.csv',encoding='cp949')
del df['Unnamed: 0.1']
df.shape

(116377, 6)

## 2. Preprocessing  

#### 1) 영화 title 찾기, 영화 배우 찾는 질문 삭제

In [397]:
# find out movie title
movie_title=[]
for i in range(len(df)):
    a=df.answer[i].split('\r')[0]
    movie_title.append(a)

# change answer to movie title
df.answer=movie_title

# delete questions not for find out movie title
for i in range(len(df)):
    a=df.answer[i].find('영화배우')
    if a!=-1:
        not_movie_title.append(i)

df.drop(not_movie_title,axis=0,inplace=True)
df.shape

(116314, 6)

#### 2) 관련 장르 이외 제외, na값 제거, Questinon 길이 50미만 제거, answer 길이 25 이상 제거, 질문 중복 제거

In [398]:
# drop gen
df['q_gen']=df['q_gen'].fillna('없음')
drop_gen=list(df.q_gen.value_counts().index)[10:]

for gen in drop_gen:
    df=df[df.q_gen!=gen]

# delete no answer
df=df[df.answer!='           ']
df=df[df.answer!='   ']
df=df[df.answer!='  ']
df=df[df.answer!=' ']
df=df[df.answer!='  관련 장면 :']
df=df[df.answer!='관련 장면 :']
df=df[df.answer!='혹시 이 영화가 아닐까요.']
df=df[df.answer!='혹시 이 영화 아닐까요.']
df=df[df.answer!='혹시 이 영화가 아닐까 싶은데요.']
df=df[df.answer!='혹시 이 영화가 아닐까 싶습니다.']
df=df[df.answer!='찍어봅니다.']
df=df[df.answer!='혹시 이 영화일까요.']
df=df[df.answer!='혹시 이 영화가 아닐까 싶네요.']

# delete no synopsis 
df=df[df.synopsis!='There is no synopsis']

# delete nan synopsis, question
df=df.dropna(subset=['synopsis','question'])

# find out question length and delete <50
len_question=[]
for q in df.question:
    len_question.append(len(q))
    
df['len_question']=len_question
df=df[df.len_question>=50]

# drop answers length>25
len_answer=[]
for a in df.answer:
    len_answer.append(len(a))
    
df['len_answer']=len_answer
df=df[df.len_answer<=25]

# 빈 공간 제거
df.loc[:,'answer']=df.loc[:,'answer'].str.strip()

# drop duplicates
df=df.drop_duplicates(subset='question')

# reindex
df=pd.DataFrame.reset_index(df,drop=True)

df.shape

(40394, 8)

#### 2-1) 총 다른 영화 질문 개수 

In [399]:
# 중복 등장한 횟수로 새로운 변수 생성 
count_dict=dict(df.answer.value_counts())
df['count']=df['answer'].map(count_dict)

# 한 번만 등장한 영화 재거
df=df[df['count']>1]

df.shape

(38614, 9)

In [400]:
# find out same movie title count
s=df.answer.value_counts()
len(s)

3788

In [404]:
# n 개 이상의 질문이 있는 영화의 수
sum(df.answer.value_counts().values>=2)

3788

#### 2-2) 새로운 인덱싱

In [402]:
# 새롭게 인덱스 매기기
key=set(list(df.answer))
value=list(range(len(key)))
index_dictionary = dict(zip(key, value))

df['new_index']=df['answer'].map(index_dictionary)

# reindex
df=pd.DataFrame.reset_index(df,drop=True)

#### 3) df로 저장

In [403]:
pd.DataFrame.to_csv(df,'formodeling.csv',encoding='cp949')